In [2]:
import openai
import subprocess
from urllib.request import urlopen
import os
import fal_client
import re
import dotenv

In [3]:
dotenv.load_dotenv()

True

## AI Modules

In [ ]:

class GPT4():
    def __init__(self, prompt=''):
        self.client = openai.AzureOpenAI(
                            azure_endpoint=os.environ['OPENAI_URL'],  # Replace with your endpoint
                            api_key=os.environ['OPENAI_KEY'],  # Replace with your key
                            api_version="2024-02-15-preview"
                            )
        self.model = 'gpt-4o'
        self.prompt = prompt
    
    def ask(self, question):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{'role':'system','content':self.prompt},
    {
        'role':'user',
        'content':[{'type':'text', 'text':question}]}],
            
        )
        return response.choices[0].message.content
    


In [40]:
def on_queue_update(update):
    if isinstance(update, fal_client.InProgress):
        for log in update.logs:
            print(log["message"])

In [41]:


class FalClient():
    def __init__(self, model='fal-ai/flux/dev'):
        self.model = model
    
    def prompt(self, prompt):
        res = fal_client.subscribe(
                self.model,
                arguments={
                    "prompt": f'{prompt}',
                },
                with_logs=True,
                on_queue_update=on_queue_update,
            )
        return res




In [42]:

class FalVidClient():
    def __init__(self, model='fal-ai/fast-svd-lcm'):
        self.model = model
    
    def prompt(self, prompt):
        res = fal_client.subscribe(
                self.model,
                arguments={
                    "image_url": prompt,
                },
                with_logs=True,
                on_queue_update=on_queue_update,
            )
        return res


    def on_queue_update(update):
        if isinstance(update, fal_client.InProgress):
            for log in update.logs:
                print(log["message"])


In [43]:
gpt = GPT4()
fal  = FalClient()
vidclient = FalVidClient()

## Constants

In [6]:
words_num = 150
scene_num = 10
segment_num = scene_num*2
id=0

In [7]:
# Story refine
story2scene_step1_prompt = f"Check the provided story, ensuring it encompasses approximately {words_num} words. If the narrative falls short, enrich it by elaborating on character interactions, incorporating dialogues(quoted directly with double quotes), actions, and emotional expressions. For the sake of simplicity and understandability of the story, do not add unnecessary characters. The total number of characters should ideally not exceed three, and group characters should not appear in the story (such as neighbors, villagers). The plot's continuity and connections should also be fortyfied. Then several adjustments should be made. Firstly, adjust the naming of characters in the story in a way that preserves the original names, refining the approach to character naming without altering them. Note that changing Jimmy to Tom is considered an alteration, while changing Mr. Green(Tom's father) to Tom's father Mr. Green is not regarded as a change. Make the adjustments according to the rule below: A human character in the story should be choosed as the protagonist, and he should simply be referred to as his name. For example, Tom. Other human characters related to the protagonist should be named by their relationship to the protagonist, such as 'Tom's father Mr. Green', 'Tom's brother Jack', Tom's friend Kitty, Tom's friend Lucy and so on. For animal characters, use the kind followed by the specific name to name them, for example, 'The cat Jimmy, The tiger Wudy.' No punctuation is required between any name. The naming rules above must be strictly obeyed. So an example of characters for a story can be like: 'Tom's father Mr. Green, Tom's brother Jack, Tom's neighbour Mrs. Brown, Tom's friend Kitty, Tom's friend Lucy, The cat Jimmy, The tiger Wudy.' If there is no human character in the story, just follow the naming rules for animal characters. Whenever referring to a character in the story, obey the naming rules above. Secondly, for any object in the story, always name it with its color and shape, such as 'the red round ball, a blue toy car, etc.' Whenever referring to the object, use its full name containing the color and shape. Finally, Note that The story is aimed at a children's audience, so be mindful to avoid using uncommon depictions of character actions, environmental settings, and the like. Replace rare and complex animal characters in the story with animals from similar species common in daily life, such as cat, dog, bird, pig, tiger, wolve, bear, lion, monkey, etc. For example, If the character in the story is a porcupine, replace it with a pig. You should directly give the new version of the story without any other additional words. The original story is enclosed in three single quotes:\n"

# Chars and settings
story2scene_step2_prompt_1 = "You will be given a story enclosed in three single quotes:\n"
story2scene_step2_prompt_2 = "\nI'm working on a vlog that narrates the story and need detailed descriptions for all characters and settings involved.\nCharacters can be devided into human characters and animal characters, and you should consider any man or animal that makes multiple appearances within the narrative as a character. The total number of characters should not exceed three, and group characters should not appear in the story (such as neighbors, villagers). Every character should be described separately, even if he is part of a group like a family or one's neighbors. The general groups that are not introduced individually should not be regarded as characters. When naming characters in the story, stricly adhere to the rule: ###A human character in the story should be choosed as the protagonist, and he should simply be referred to as his name. For example, Tom. Other human characters related to the protagonist should be named by their relationship to the protagonist, such as 'Tom's father Mr. Green', 'Tom's brother Jack', Tom's friend Kitty, Tom's friend Lucy and so on. For animal characters, use the kind followed by the specific name to name them, for example, 'The cat Jimmy, The tiger Wudy.' No punctuation is required between any name.### The naming rules above must be strictly obeyed. So an example of characters for a story can be like: 'Tom's father Mr. Green, Tom's brother Jack, Tom's neighbour Mrs. Brown, Tom's friend Kitty, Tom's friend Lucy, The cat Jimmy, The tiger Wudy.' If there is no human character in the story, just follow the naming rules for animal characters. Description comes after the name: For human characters, please format character descriptions as follows: 'Name: A brief introduction. detailed description, including hair, eyes, face shape, and clothing details (color + shirt, trousers, shoes). Clothes should be in solid colors and standard styles.' For example, 'Tom: An 8-year-old boy, curious and compassionate. Round face, short brown hair, wide blue eyes, wearing a red T-shirt, blue trousers, and red sneakers. Tom's father Mr. Green: A calm and thoughtful man. Square jaw, dark short-cut hair, deep-set brown eyes, wearing a plain white shirt, blue jeans, and brown boots.' For animal characters, please format character descriptions as follows: 'Name: A brief introduction (end with the animal kind). Major characteristics, especially fur color and eye color (solid color preferred).' For example, 'The cat Jimmy: A vibrant cat, with sleek orange fur, piercing green eyes, and a narrow, angular face. The pirrot Jerry: A smart parrot, with bright green feather and black eyes.' Note that description for a character should be simple and clear, and in 30 words.\nAs for settings in the story, avoid including any humans or animals or their activities in the descriptions, such as 'a place where kids play'. The background color in the setting should be distinctly different from the clothing and skin tone of the characters, with a strong contrast. For example, since the characters have flesh-colored skin, yellow should not be used as the background color. For those settings with buildings, avoid using overly broad distant concepts, such as 'village (outdoor)' or 'school (outdoor)'. You should format setting descriptions as follows: 'Name (Indoor/Outdoor): A brief introduction of the major elements in simple styles and colors.' Specify whether the setting is indoor or outdoor. For example, 'Tom's House (Outdoor): A cozy, single-story with a small porch. Blue walls, large window on one side.' Note that description for a setting should be simple and clear, and in 30 words.\nYour answer should be in the format:\n'Characters list:[A, B, C, ......]\nSettings list:[A, B, C, ......]\nPart 1. Characters:\n......\nPart 2. Settings:\n......'\nAn example is:\n'Characters list:[Tom, Tom's father Mr. Green, Tom's neighbour Mrs. Brown, Tom's friend Jack, Tom's friend Lucy, The cat Jimmy, The tiger Wudy]\nSettings list:[Tom's bedroom (Indoor), Tom's house (Outdoor), Neighbourhood (Outdoor)]\nPart 1. Characters:\nTom: An 8-year-old boy, curious and compassionate. Round face, short brown hair, wide blue eyes, wearing a red T-shirt, blue trousers, and red sneakers.\nTom's father Mr. Green:......\n......\nThe tiger Wudy: A lively tiger, with orange fur and black eyes.\nPart 2. Settings:\n......\nTom's House (Outdoor): A cozy, single-story with a small porch. Blue walls, large window on one side.\n......'"

# Scenes
story2scene_step3_prompt_1 = "The following is a story enclosed in three single quotes:\n"
story2scene_step3_prompt_2 = "I'm working on a vlog that narrates the story. The characters and settings for the story has been determined as below:\n"
story2scene_step3_prompt_3 = f"Following this, I need a script divided into {scene_num} scenes, with each scene thoroughly detailing characters (including dialogues, actions, and emotions) and settings. The number of characters in a scene should not exceed two. In other words, one scene can never include up to three characters. Format the scene descriptions with '[All Characters Included][Setting Included] Description of the scene.' The character list you can choose for the scenes are: "
story2scene_step3_prompt_4 = "The setting list you can choose for the scenes are: "
story2scene_step3_prompt_5 = f"Always use characters' original names in the scene description, and avoid using pronouns for characters within scenes. Remember: Ensure every character named in the scene is listed in the first bracket. Besides, for any object in the story, always name it with its color and shape, such as 'the red round ball, a blue toy car, etc.' Whenever referring to the object, use its full name containing the color and shape. For example,\n'Scene 1: [Jack, Jack's father Mr. Green, The cat Jerry][Living Room (Indoor)] Jack and Jack's father is reading a green book concentrately, where the cat Jerry is sleeping in the corner.\nScene 2: ......'\nJust as the example shows, every human character and animal character included in the scene is listed in the first [], and always referred to by their full name. The object book is also described with the word 'green'. Follow the formats strictly, your answer should only involve full descriptions of {scene_num} scene breakdowns. Directly give your answer with no additional explanation."

# Correctness
story2scene_step4_prompt_1 = "The following is a drafted script enclosed in three single quotes:\n"
story2scene_step4_prompt_2 = "\nStep1. Review the content within the square brackets scene by scene in Part 'Scenes', focusing on the characters(indicated in the first set of brackets) and settings(in the second set). If any character or setting isn't among the names listed in Part 'Characters' or 'Settings', you should answer: 'Yes.'. Even terminology inconsistencies must be strictly recognized.\nStep2. Review the content scene by scene in Part 'Scenes', ensuring every character who needs to be represented in the scene's illustration is encompassed within the initial set of brackets. If a character is missing from the first set of brackets, you should answer: 'Yes.'. If no problem is found in the two steps above, answer: 'No problem found.' Your answer should begin with 'Yes.' or 'No problem found.' and then your explanations. Afer your explanations, if your answer is 'Yes.', please supply a revised version of the script at last, maintaining the original format and starting with: '###New Version of Complete Script###'. All the text following '###New Version of Complete Script###' should be identical to the input script, except for the modifications you propose."

# Segment
scene2segment_step1_prompt_1 = "The following is a script enclosed in three single quotes:\n"
scene2segment_step1_prompt_2 = f"\nThere are {scene_num} scenes in the part 'Scenes' in the script above. You need to devide each of the {scene_num} scenes into two segments(thus {segment_num} segments in total): the beginning of the scene and the end of the scene. The description for these two segments should be relatively independent but clearly linked in terms of time progression. Every segment should accurately reflect the characters' speech, behavior, and actions. Include direct quotes for any dialogue, and ensure all spoken words are presented verbatim. If the original dialogue is incomplete, prioritize its completion. Because we want the AI tool to generate diagrams based on the segment description, it's preferable to simplify the actions and interactions between characters, avoiding overly complex or difficult-to-illustrate descriptions. Place anything about the lens transition in the last parentheses. Besides, for any object in the story, always name it with its color and shape, such as 'the red round ball, a blue toy car, etc.' Whenever referring to the object, use its full name containing the color and shape.\nCraft descriptions for each segment adhering to the specified format: Scene x Segment y: [Characters][Setting] Segment content (camera directions). For instance, \"Scene 2 Segment 1: [Tommy][living room] Tommy is sitting and playing with toys intensively in the living room. Suddenly, a loud crash is heard from outside. Tommy is startled by the sudden noise (The camera focuses from the far view onto Tommy's face.)\" Note that all the {segment_num} segments should be listed completely. Directly give your answer beginning with: 'Scene 1 Segment 1:' without any additional words. A selected excerpt from an example is: '......Scene 3 Segment 1: [Jimmy, Jimmy's Father][Jimmy's House (Outdoor)] Jimmy approaches his father, who is inspecting the broken window with a calm demeanor. \"What happened here, Dad?\" Jimmy asks, concern lacing his voice. His father turns, offering a smile that\u2019s both comforting and mystified, \"I was hoping you might tell me, buddy.\" (Close-up on their faces, showing a mixture of concern and curiosity.)\nScene 3 Segment 2: [Jimmy, Jimmy's Father][Jimmy's House (Outdoor)] Their conversation deepens the mystery but also shows their mutual resolve to fix the window and find out what caused the break. \"We'll figure this out together,\" his father assures him. Jimmy nods, his expression firming with resolved determination (The camera pulls back to frame them side-by-side, looking at the broken window.)\n......' Your answer should contain the complete {segment_num} segments following the format given in the example. Any omission is not allowed."

# Recheck segments for consistency
scene2segment_step2_prompt_1 = "The following is a drafted script enclosed in three single quotes:\n"
scene2segment_step2_prompt_2 = f"\nStep1. Check if the script is missing any segments, specifically if there are fewer than {segment_num} segments. If any segment is missing from the script, you should answer: 'Yes.'. Step2. Review the content within the square brackets segment by segment in Part 'Scenes', focusing on the characters(indicated in the first set of brackets) and settings(in the second set). If any character or setting isn't among the names listed in Part 'Characters' or 'Settings', you should answer: 'Yes.'. Even terminology inconsistencies must be strictly recognized.\nStep3. Review the content segment by segment in Part 'Scenes', ensuring every character who needs to be represented in the segment's illustration is encompassed within the initial set of brackets. If a character is missing from the first set of brackets, you should answer: 'Yes.'. If no problem is found in the three steps above, answer: 'No problem found.' Your answer should begin with 'Yes.' or 'No problem found.' and then your explanations. Afer your explanations, if your answer is 'Yes.', please supply a revised version of the script at last, maintaining the original format and starting with: '###New Version of Complete Script###'. All the text following '###New Version of Complete Script###' should be identical to the input script, except for the modifications you propose."

# Character layout for each segment
scene2segment_step3_prompt_1 = f"The content in the first set of brackets in each of the {segment_num} segments represents the characters included, but not 100% correct. Now I want to draw an illustration for each segment. So determine which character should be shown in the illustration and modify the content in the first [] for each of the {segment_num} segments. The order of characters in the first [] is also very important. If in previous segments, the order of characters is [A, B], unless there's explicit mention of a character position exchange in the narrative, the order should remain [A, B], not [B, A], in the current segment. Any character listed must be strictly among the Characters list:"
scene2segment_step3_prompt_2 = "For example, if 'Tom's sister Lily' is in the Characters list, even 'Tom's sister' or 'Lily' is not allowed in your answer. Directly give your answer with no additional words, following the format of:\nScene 1 Segment 1: [A, B, ...]\nScene 1 Segment 2: [...]\n..."

# Recheck character's n each segment and ensure consistency with the setting
scene2segment_step4_prompt = f"The content in the first set of brackets in each of the {segment_num} segments represents the characters list included. Now, I want to replace all the pronouns in the Segment Description with the names from the character list included in the segment, without changing anything else. For example, for Scene 4 Segment 2: [Max, Lily, Max’s mother, The cardinal Red][The park (Outdoor)] As they watch, Red flutters its wings, a live painting against the greenery. Their mother joins in, 'Beautiful, isn’t it?' Their heads nod in unison, captivated by the vibrant splash of red. (The camera zooms in on the cardinal, then performs a gentle fade out with the bird taking flight.) should be changed to Scene 4 Segment 2: [Max, Lily, Max’s mother, The cardinal Red][The park (Outdoor)] As Max and Lily watch, The cardinal Red flutters its wings, a live painting against the greenery. Max’s mother joins in, 'Beautiful, isn’t it?' Max and Lily heads nod in unison, captivated by the vibrant splash of The cardinal Red. (The camera zooms in on The cardinal Red, then performs a gentle fade out with The cardinal Red taking flight.) Your response should keep the same format as the input, such as：Scene 1 Segment 2: [Max, Lily][The park (Outdoor)] Their laughter fills the air as they swing higher, reaching the peak of their arcs. Their mother watches with a warm smile, enjoying the simple joy of the moment (The camera focuses on their mother's smiling face, then pans to the sky, capturing the swings in motion.)\nScene 2 Segment 1: [Max, Lily][The park (Outdoor)] Racing towards the slide, their laughter rings through the park. Max reaches the top first, turning to wait for Lily. 'Hurry up!' he calls down, eager for the shared thrill. (Close-up on Max’s excited face as he looks down the slide.)...... Note that your answer should contain and only contain full description of {segment_num} segments."

# Convert seg to prompt for SD3
segment2prompt_prompt_1 = "The following is a script enclosed in three single quotes:\n"
segment2prompt_prompt_2 = "\nI'd like to create an illustration for the plot of each segment. Please provide a clear visual description based on the content of the segment to facilitate the creation of the illustration. Since the illustration is static, do not include content such as camera movements, the content of conversations or any other elements that cannot be depicted in a single illustration. Because we want the AI tool to generate diagrams based on the segment description, it's preferable to simplify the actions and interactions between characters, avoiding overly complex or difficult-to-illustrate descriptions. In your visual description after the two set of brackets [], All character involved should be renamed following the rule below: 'Characters can be devided into Human Characters and Animal Characters. Each Human Character involved should be referred to by name and the color of their top, such as Jimmy (in red), Jimmy’s father (in grey), and so on. The color of a human character's top should be fetched from the 'Characters' part. Each Animal Character involved should be referred to by the animal kind and the name, such as The cat Kitty, The bird Betty, and so on. The animal kind of an animal character should also be fetched from the 'Characters' part. Avoid just using pronouns for characters within discription.' Note, any content in the two set of brackets [] of each segment are fixed in the renaming process. Besides, for any object in the story, always name it with its color and shape, such as 'the red round ball, a blue toy car, etc.' Whenever referring to the object, use its full name containing the color and shape. Besides, the description for the two segments under one scene should be relatively independent but clearly linked in terms of time progression.\n"
segment2prompt_prompt_3 = f". For characters who do not need to appear in the illustration, remove them from the first set of brackets [] in each segment. Conversely, for characters omitted from the first set of brackets [] in a segment but who should appear in the illustration, add them to the first set of brackets [] in that segment. An important rule judging whether a character should be included is: Whether the character is depicted in the segment description after the two set of brackets []. Note that any content in the [] should never be renamed. For example, even if the character list is: [Jimmy's dog Max, The cat], they can't be renamed as [The dog Max, The cat Kitty]. All characters chosen should be strictly among the character list given. Your response should keep the same format as the input, such as：Scene 1 Segment 1: [Max, Lily][The park (Outdoor)] Max(in red) and Lily（in pink）'s laughter fills the air as they swing higher, reaching the peak of their arcs. Their mother(in green) watches with a warm smile, enjoying the simple joy of the moment\nScene 1 Segment 2:...... Note that your answer should contain and only contain full description of {segment_num} segments."

## Part one: Story Refinement

In [48]:
prompt = """
Make stories from the curious questions the user has also answering his question, ensuring it encompasses approximately 150 words. If the narrative falls short, enrich it by elaborating on character interactions, incorporating dialogues(quoted directly with double quotes), actions, and emotional expressions. For the sake of simplicity and understandability of the story, do not add unnecessary characters. The total number of characters should ideally not exceed three, and group characters should not appear in the story (such as neighbors, villagers). The plot's continuity and connections should also be fortyfied. Then several adjustments should be made. Firstly, adjust the naming of characters in the story in a way that preserves the original names, refining the approach to character naming without altering them. Note that changing Jimmy to Tom is considered an alteration, while changing Mr. Green(Tom's father) to Tom's father Mr. Green is not regarded as a change. Make the adjustments according to the rule below: A human character in the story should be choosed as the protagonist, and he should simply be referred to as his name. For example, Tom. Other human characters related to the protagonist should be named by their relationship to the protagonist, such as 'Tom's father Mr. Green', 'Tom's brother Jack', Tom's friend Kitty, Tom's friend Lucy and so on. For animal characters, use the kind followed by the specific name to name them, for example, 'The cat Jimmy, The tiger Wudy.' No punctuation is required between any name. The naming rules above must be strictly obeyed. So an example of characters for a story can be like: 'Tom's father Mr. Green, Tom's brother Jack, Tom's neighbour Mrs. Brown, Tom's friend Kitty, Tom's friend Lucy, The cat Jimmy, The tiger Wudy.' If there is no human character in the story, just follow the naming rules for animal characters. Whenever referring to a character in the story, obey the naming rules above. Secondly, for any object in the story, always name it with its color and shape, such as 'the red round ball, a blue toy car, etc.' Whenever referring to the object, use its full name containing the color and shape. Finally, Note that The story is aimed at a children's audience, so be mindful to avoid using uncommon depictions of character actions, environmental settings, and the like. Replace rare and complex animal characters in the story with animals from similar species common in daily life, such as cat, dog, bird, pig, tiger, wolve, bear, lion, monkey, etc. For example, If the character in the story is a porcupine, replace it with a pig. You should directly give the new version of the story without any other additional words.The original story is enclosed in three single quotes:\n
"""

story = "Why are differential equations a graph"
carGpt = GPT4(prompt=prompt)
story_refined = carGpt.ask("'''" + story + "'''")
story_refined

'Tom was sitting at his desk, staring at a page filled with squiggly lines and strange symbols. "Why are differential equations like a graph?" he wondered aloud, tapping his pencil against his notebook. \n\nTom\'s friend Lucy, holding a green round apple, walked into the room and saw his puzzled face. "What\'s wrong, Tom?" she asked, taking a bite of the green round apple.\n\n"These differential equations," Tom replied, pointing to his book. "I keep hearing they\'re like graphs, but I don’t get it!"\n\nLucy sat beside him and smiled. "Don\'t worry, it’s not that complicated. Imagine you’re drawing dots for every moment of time when something is happening, like how fast the wind blows. Those dots are like the graph\'s path, and a differential equation is the rule that connects the dots! It tells you how one thing changes with respect to something else."\n\nTom’s eyes lit up. "So, it’s telling a story about change?"\n\n"Exactly!" Lucy said, finishing the green round apple. "It’s like a g

## Part two: Story Generation

In [11]:
charactrGpt_prompt = story2scene_step2_prompt_1 + "'''" + story_refined + "'''" + story2scene_step2_prompt_2

In [12]:
char_genGpt = GPT4(prompt=story2scene_step2_prompt_1+story2scene_step2_prompt_2)
character_story = char_genGpt.ask("'''" + story_refined + "'''")

I have taken the output from cell 9&10 and fed them into cell 11

In [13]:
characters_dict, settings_dict = {}, {}
characters_list, settings_list = [], []
story2scene_step2_question = story2scene_step2_prompt_1 + "'''" + story_refined + "'''" + story2scene_step2_prompt_2
print(f"Story2Scene Step2 Question:\n{story2scene_step2_question}\n")
while(characters_dict == {} or settings_dict == {}):
        story2scene_step2_answer = character_story
        print(f"Story2Scene Step2 Answer:\n{story2scene_step2_answer}\n")
        try:
            characters_list = story2scene_step2_answer.split("Characters list:")[1].split("Settings list:")[0].strip().strip("[]").split(", ")
            settings_list = story2scene_step2_answer.split("Settings list:")[1].split("Part 1.")[0].strip().strip("[]").split(", ")
            if not characters_list or not settings_list:
                raise ValueError("Extraction of characters or settings names failed.")
            characters_dict = {name: "" for name in characters_list}
            settings_dict = {name: "" for name in settings_list}
            characters_description = story2scene_step2_answer.split("Part 1. Characters:")[1].split("Part 2.")[0].strip()
            for character in characters_list:
                start = characters_description.find(character + ":") + len(character) + 2
                end = characters_description.find("\n", start)
                description = characters_description[start:end].strip()
                if not description:
                    raise ValueError(f"Description extraction failed for character: {character}")
                characters_dict[character] = description
            # Extract descriptions for settings
            settings_description = story2scene_step2_answer.split("Part 2. Settings:")[1].strip()
            for setting in settings_list:
                start = settings_description.find(setting + ":") + len(setting) + 2
                end = settings_description.find("\n", start)
                description = settings_description[start:end].strip()
                if not description:
                    raise ValueError(f"Description extraction failed for setting: {setting}")
                settings_dict[setting] = description
            # Ensure all keys have been filled
            if not all(characters_dict.values()) or not all(settings_dict.values()):
                raise ValueError("Not all descriptions were successfully extracted.")
        except Exception as e:
            characters_dict, settings_dict = {}, {}

Story2Scene Step2 Question:
You will be given a story enclosed in three single quotes:
'''Sherlock was sitting under the big green oak tree, gazing up at the blue sky. His loyal dog Toby sat by his side, wagging his tail. The dog Toby tilted his head, as if trying to decipher what Sherlock was thinking.

"You know, Toby," Sherlock said with a smile, "the clouds tell stories too. Do you see that one over there? It looks like a giant fish."

The dog Toby barked softly, as if agreeing. Sherlock chuckled, reaching out to ruffle the dog Toby's fur. "You're a good listener, old friend."

Just then, Sherlock’s sister Lucy came running with the red round ball in her hands. "Sherlock, catch!" she exclaimed, tossing the red round ball. It landed near the dog Toby, who immediately jumped up and sniffed at it.

Sherlock smiled warmly. "Alright, Lucy, let's play." With his sister Lucy and the dog Toby, Sherlock spent the rest of the day enjoying the beautiful weather and the stories written in the 

So settings and characters extraction is complete

In [14]:
scene_prompt = story2scene_step3_prompt_1 + "'''" + story_refined + "'''\n" + story2scene_step3_prompt_2 + 'Characters:\n' + str(characters_dict) + '\nSettings:\n' + str(settings_dict) + '\n' + story2scene_step3_prompt_3 + str(characters_list) + '. ' + story2scene_step3_prompt_4 + str(settings_list) + '. ' + story2scene_step3_prompt_5

In [15]:
scenegpt = GPT4(prompt='Follow the instruction the user gave you, You are a visual intelligence assistant.')
scene_resp = scenegpt.ask(scene_prompt)
scene_resp

'Scene 1: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)]  \nSherlock is sitting under the big green oak tree, gazing up at the bright blue sky. Sherlock\'s oval face is calm and thoughtful as he leans back against the sturdy trunk. Sherlock is wearing a green shirt, gray trousers, and black shoes. The dog Toby sits loyally next to Sherlock, wagging his tail slowly. The dog Toby, with soft brown fur and gentle dark brown eyes, tilts his head while looking at Sherlock, as if trying to decipher Sherlock\'s thoughts.  \n\nScene 2: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)]  \nSherlock runs his fingers through the dog Toby\'s soft brown fur and smiles. Sherlock points to a fluffy white cloud drifting in the bright blue sky. "You know, Toby," Sherlock says with a smile, "the clouds tell stories too. Do you see that one over there? It looks like a giant fish." The dog Toby barks softly, his tail wagging slightly as if signaling his agreement. Sherlock laughs lightly, gaz

In [16]:
scene_formatted = '\n'.join([scene for scene in scene_resp.split('\n') if scene.strip('') != ''])

In [17]:
characters_str = '\n'.join([f"{key}: {value}" for key, value in characters_dict.items()])
settings_str = '\n'.join([f"{key}: {value}" for key, value in settings_dict.items()])
script = 'Characters:\n' + characters_str + '\nSettings:\n' + settings_str + '\nScenes:\n' + scene_formatted 

In [18]:

reviewgpt = GPT4(prompt='Follow the instruction the user gave you, You are a visual intelligence assistant.')
story2scene_step4_answer = ""
story2scene_step4_redo = 0
while ("No problem found".lower() not in story2scene_step4_answer.lower()) and (story2scene_step4_redo <= 3):
    if "New Version of Complete Script".lower() in story2scene_step4_answer.lower():
        matches = re.search("New Version of Complete Script.*?Characters:(.*)", story2scene_step4_answer, re.DOTALL)
        if matches:
            script = 'Characters:\n' + matches.group(1).strip().strip("'").strip()
        else:
            break
    elif story2scene_step4_redo != 0:
        print("No New Version of Complete Script")
        break
    story2scene_step4_question = story2scene_step4_prompt_1 + "'''" + script + "'''" + story2scene_step4_prompt_2
    story2scene_step4_answer = reviewgpt.ask(story2scene_step4_question)
    key = f'step4_answer_redo{story2scene_step4_redo}' if story2scene_step4_redo > 0 else 'step4_answer'
    reviewed = story2scene_step4_answer
    print(f"Story2Scene Step4 Answer:\n{story2scene_step4_answer}\n")
    story2scene_step4_redo += 1
lines = script.split('\n')
for i, line in enumerate(lines):
    line = line.rstrip()
    if line and line[-1] not in '.!?,:;':
        lines[i] = line + '.'
script = '\n'.join(lines)

Story2Scene Step4 Answer:
No problem found. 

Explanation:  
1. **Step 1:** Reviewing scene by scene:  
   - The characters and settings within the square brackets match the names listed in Part 'Characters' and 'Settings'. There are no inconsistencies or missing elements in terminology or naming conventions.  

2. **Step 2:** Ensuring all characters present in the scene descriptions are included in the first set of brackets:  
   - Each character explicitly described in the scene is present in the corresponding first set of brackets. No missing characters are found.  

As both steps have been reviewed without any issues, the script is consistent and complete.



## Final Script

In [19]:
script

'Characters:\nSherlock: A thoughtful boy who enjoys the outdoors and storytelling. Oval face, short black hair, dark brown eyes, wearing a green shirt, gray trousers, and black shoes.\nSherlock\'s sister Lucy: An energetic girl with a playful spirit. Round face, long brown hair tied in a ponytail, lively hazel eyes, wearing a pink shirt, blue trousers, and white sneakers.\nThe dog Toby: A loyal dog, with soft brown fur and gentle dark brown eyes.\nSettings:\nThe oak tree meadow (Outdoor): A serene meadow dominated by a big green oak tree. Lush green grass covers the ground, with a bright blue sky overhead.\nScenes:\nScene 1: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)].\nSherlock is sitting under the big green oak tree, gazing up at the bright blue sky. Sherlock\'s oval face is calm and thoughtful as he leans back against the sturdy trunk. Sherlock is wearing a green shirt, gray trousers, and black shoes. The dog Toby sits loyally next to Sherlock, wagging his tail slowly. T

## Get Scenes

In [20]:
scene_numbers = [int(num) for num in re.findall(r"Scene (\d+)", script)]
scene_num = max(scene_numbers) if scene_numbers else 0
segment_num = scene_num * 2
print(scene_num, segment_num)

10 20


### Divide by 2

In [21]:
segment_prompt = scene2segment_step1_prompt_1 + "'''" + script + "'''" + scene2segment_step1_prompt_2

In [22]:
segmentgpt = GPT4(prompt='Follow the instruction the user gave you, You are a visual intelligence assistant.')
segmentresp = segmentgpt.ask(segment_prompt)
segmentresp

'Scene 1 Segment 1: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)]  \nSherlock is sitting under the big green oak tree, leaning back against its sturdy trunk with a calm and thoughtful expression. Sherlock\'s oval face reflects serenity as he gazes up at the bright blue sky. The dog Toby is seated loyally beside him, wagging his tail slowly and tilting his head as though trying to understand Sherlock’s quiet musings (The camera focuses on Sherlock\'s face and the gentle movement of the dog Toby).  \n\nScene 1 Segment 2: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)]  \nThe dog Toby lowers his head and rests it on his paws while Sherlock remains silent under the lush canopy of the tree. The peaceful moment lingers as the soft rustling of leaves gently animates the serene scene (The camera pans upward to show the bright blue sky through the oak tree’s branches).  \n\nScene 2 Segment 1: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)]  \nSherlock runs his fingers 

In [23]:
segmentformatted = '\n'.join([line for line in segmentresp.split('\n') if line.strip() != ''])

In [24]:
part1_part2_content = re.search(r'(Characters.*?Scenes)', script, re.DOTALL) 
part1_part2_content = part1_part2_content.group(0)
part1_part2_content = re.sub(r'^(Characters|Settings|Scenes)$', r'\1:', part1_part2_content, flags=re.MULTILINE) # Add a ':' after "Characters", "Settings" or "Scenes" on a separate line
scene2segment_step1_answer = part1_part2_content + "\n" + segmentformatted
scene2segment_step1_answer = re.sub(r'^\s*$\n', '', scene2segment_step1_answer, flags=re.MULTILINE)

In [25]:
scene2segment_step1_answer

'Characters:\nSherlock: A thoughtful boy who enjoys the outdoors and storytelling. Oval face, short black hair, dark brown eyes, wearing a green shirt, gray trousers, and black shoes.\nSherlock\'s sister Lucy: An energetic girl with a playful spirit. Round face, long brown hair tied in a ponytail, lively hazel eyes, wearing a pink shirt, blue trousers, and white sneakers.\nThe dog Toby: A loyal dog, with soft brown fur and gentle dark brown eyes.\nSettings:\nThe oak tree meadow (Outdoor): A serene meadow dominated by a big green oak tree. Lush green grass covers the ground, with a bright blue sky overhead.\nScenes:\nScene 1 Segment 1: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)]  \nSherlock is sitting under the big green oak tree, leaning back against its sturdy trunk with a calm and thoughtful expression. Sherlock\'s oval face reflects serenity as he gazes up at the bright blue sky. The dog Toby is seated loyally beside him, wagging his tail slowly and tilting his head as t

## Check it for consistency


In [26]:

segment2gpt = GPT4(prompt='Follow the instruction the user gave you, You are a visual intelligence assistant.')
final_flag = 0
retry_times = 0
current_answer = scene2segment_step1_answer
# scene2segment step2 ---check the script
scene2segment_step2_answer = ""
scene2segment_step2_redo = 0
while ("No problem found".lower() not in scene2segment_step2_answer.lower()) and (scene2segment_step2_redo <= 3):
    if "New Version of Complete Script:".lower() in scene2segment_step2_answer.lower():
        matches = re.search("New Version of Complete Script.*?Characters:(.*)", scene2segment_step2_answer, re.DOTALL)
        if matches:
            current_answer = 'Characters:\n' + matches.group(1).strip().strip("'").strip()
    elif scene2segment_step2_redo != 0:
        print("No New Version of Complete Script")
        break
    scene2segment_step2_question = scene2segment_step2_prompt_1 + "'''" + current_answer + "'''" + scene2segment_step2_prompt_2
    print(f"Scene2Segment Step2 Question:\n{scene2segment_step2_question}\n")
    scene2segment_step2_answer = segment2gpt.ask(scene2segment_step2_question)
    key = f'step2_answer_redo{scene2segment_step2_redo}' if scene2segment_step2_redo > 0 else 'step2_answer'
    print(f"Scene2Segment Step2 Answer:\n{scene2segment_step2_answer}\n")
    scene2segment_step2_redo += 1

print(f"Scene2Segment Intermediate Answer:\n{current_answer}")
        
# scene2segment step3 ---specify the character layout of each segment
print(f"******************No.{id} Scene2Segment Step3 Begin******************")
index = current_answer.find("Scenes:\n")
scenes_content = current_answer[(index+8):]

scene2segment_step3_question = "Characters list: " + str(characters_list) + "\nScene description:\n" + scenes_content + "\n" + scene2segment_step3_prompt_1 + str(characters_list) + scene2segment_step3_prompt_2
print(f"Scene2Segment Step3 Question:\n{scene2segment_step3_question}\n")
scene2segment_step3_answer = segment2gpt.ask(scene2segment_step3_question)
scene2segment_step3_answer = '\n'.join([line for line in scene2segment_step3_answer.split('\n') if line.strip() != ''])
print(f"Scene2Segment Step3 Answer:\n{scene2segment_step3_answer}\n")
pattern = r'\[(.*?)\]'
step1_matches = re.findall(pattern, current_answer)
step2_matches = re.findall(pattern, scene2segment_step3_answer)
for i in range(0, len(step1_matches), 2):
    if i < len(step1_matches):
        escaped_match = re.escape(step1_matches[i]) 
        current_answer = re.sub(f'\\[{escaped_match}\\]', f'[{step2_matches[i//2]}]', current_answer, count=1)
print(current_answer)

# scene2segment step4 ---double-check the characters in each segment to ensure they are consistent with the character setting
print(f"******************No.{id} Scene2Segment Step4 Begin******************")
scene2segment_step4_question = f"The full description of the {segment_num} segments is:\n" + scenes_content + "\n" + scene2segment_step4_prompt 
print(f"Scene2Segment Step4 Question:\n{scene2segment_step4_question}\n")
scene2segment_step4_answer = segment2gpt.ask(scene2segment_step4_question)
scene2segment_step4_answer = '\n'.join([line for line in scene2segment_step4_answer.split('\n') if line.strip() != ''])
print(f"Scene2Segment Step4 Answer:\n{scene2segment_step4_answer}\n")
index = current_answer.find("Scenes:\n")
part1_part2_content = current_answer[:(index+8)]
current_answer = part1_part2_content + scene2segment_step4_answer

scene2segment_answer = current_answer # Step2-step4 is the inspection and replacement step, directly correcting the result of step1
lines = scene2segment_answer.split('\n')
for i, line in enumerate(lines):
    line = line.rstrip()
    if line and line[-1] not in '.!?,:;':
        lines[i] = line + '.'
scene2segment_answer = '\n'.join(lines)

Scene2Segment Step2 Question:
The following is a drafted script enclosed in three single quotes:
'''Characters:
Sherlock: A thoughtful boy who enjoys the outdoors and storytelling. Oval face, short black hair, dark brown eyes, wearing a green shirt, gray trousers, and black shoes.
Sherlock's sister Lucy: An energetic girl with a playful spirit. Round face, long brown hair tied in a ponytail, lively hazel eyes, wearing a pink shirt, blue trousers, and white sneakers.
The dog Toby: A loyal dog, with soft brown fur and gentle dark brown eyes.
Settings:
The oak tree meadow (Outdoor): A serene meadow dominated by a big green oak tree. Lush green grass covers the ground, with a bright blue sky overhead.
Scenes:
Scene 1 Segment 1: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)]  
Sherlock is sitting under the big green oak tree, leaning back against its sturdy trunk with a calm and thoughtful expression. Sherlock's oval face reflects serenity as he gazes up at the bright blue sky. The

In [27]:
scene2segment_answer

'Characters:\nSherlock: A thoughtful boy who enjoys the outdoors and storytelling. Oval face, short black hair, dark brown eyes, wearing a green shirt, gray trousers, and black shoes.\nSherlock\'s sister Lucy: An energetic girl with a playful spirit. Round face, long brown hair tied in a ponytail, lively hazel eyes, wearing a pink shirt, blue trousers, and white sneakers.\nThe dog Toby: A loyal dog, with soft brown fur and gentle dark brown eyes.\nSettings:\nThe oak tree meadow (Outdoor): A serene meadow dominated by a big green oak tree. Lush green grass covers the ground, with a bright blue sky overhead.\nScenes:\nScene 1 Segment 1: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)].\nSherlock is sitting under the big green oak tree, leaning back against Sherlock\'s sturdy trunk with a calm and thoughtful expression. Sherlock\'s oval face reflects serenity as Sherlock gazes up at the bright blue sky. The dog Toby is seated loyally beside Sherlock, wagging The dog Toby\'s tail sl

## Now converting segments into prompts for SD3

In [28]:
seg2prompt = segment2prompt_prompt_1 + "'''" + scene2segment_answer + "'''" + segment2prompt_prompt_2 + str(characters_list) + segment2prompt_prompt_3

In [29]:
promptgpt = GPT4(prompt='Follow the instruction the user gave you, You are a visual intelligence assistant.')
seg2prompt_ans = promptgpt.ask(seg2prompt)
print(f"Segment2Prompt Answer:\n{seg2prompt_ans}\n")
segment2prompt_answer = '\n'.join([line for line in seg2prompt_ans.split('\n') if line.strip() != ''])
# print(f"Segment2Prompt Answer:\n{segment2prompt_answer}\n")

index = scene2segment_answer.find("Scenes:\n")
part1_part2_content = scene2segment_answer[:(index+8)]
segment2prompt_answer = part1_part2_content + segment2prompt_answer
lines = segment2prompt_answer.split('\n')
for i, line in enumerate(lines):
    line = line.rstrip()
    if line and line[-1] not in '.!?,:;':
        lines[i] = line + '.'
segment2prompt_answer = '\n'.join(lines)

Segment2Prompt Answer:
Scene 1 Segment 1: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)]  
Sherlock (in green) is sitting under the big green oak tree, leaning back against its sturdy trunk with a calm and thoughtful expression. The dog Toby, with soft brown fur, is seated loyally beside Sherlock (in green), wagging his tail slowly and tilting his head slightly, adding a sense of gentle curiosity.

Scene 1 Segment 2: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)]  
Sherlock (in green) remains silent under the tree, his gaze fixed upward. Beside him, The dog Toby, with soft brown fur, lowers his head and rests it comfortably on his paws. The serene ambiance is complemented by the lush grass and the branches of the big green oak tree above.

Scene 2 Segment 1: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)]  
Sherlock (in green) sits close to The dog Toby, running his fingers gently through the dog’s soft brown fur. Sherlock (in green) uses his other hand to poi

In [30]:
pattern = r'\[(.*?)\]'
matches = re.findall(pattern, segment2prompt_answer)
character_flag = 1 
setting_flag = 1
for i in range(0, len(matches), 2):
    if i < len(matches):
        names = [name.strip() for name in matches[i].split(',')]
        if not all(name in characters_list for name in names):
            character_flag = 0
for i in range(1, len(matches), 2):
    if i < len(matches):
        places = [place.strip() for place in matches[i].split(',')]
        if not all(place in settings_list for place in places):
            setting_flag = 0
# Check if enough scenes have been generated
pattern = r'Scene (\d+)'
numbers = re.findall(pattern, segment2prompt_answer)
numbers = [int(num) for num in numbers]
current_scene_num = max(numbers) if numbers else 0
if(character_flag == 1 and setting_flag == 1 and current_scene_num >= 10):
    final_flag = 1
final_flag

0

### Splitting script into parts. for reference of script look at prompt.txt

In [32]:
parts = re.split(r'\s*(Characters:|Settings:|Scenes:)\s*', segment2prompt_answer, flags=re.IGNORECASE)
scene_part = ''
for i, part in enumerate(parts):
    if 'scenes:' in part.lower():
        if i+1 < len(parts):
            print(i,part)
            scene_part = parts[i+1]

scene_part

5 Scenes:


"Scene 1 Segment 1: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)].\nSherlock (in green) is sitting under the big green oak tree, leaning back against its sturdy trunk with a calm and thoughtful expression. The dog Toby, with soft brown fur, is seated loyally beside Sherlock (in green), wagging his tail slowly and tilting his head slightly, adding a sense of gentle curiosity.\nScene 1 Segment 2: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)].\nSherlock (in green) remains silent under the tree, his gaze fixed upward. Beside him, The dog Toby, with soft brown fur, lowers his head and rests it comfortably on his paws. The serene ambiance is complemented by the lush grass and the branches of the big green oak tree above.\nScene 2 Segment 1: [Sherlock, The dog Toby][The oak tree meadow (Outdoor)].\nSherlock (in green) sits close to The dog Toby, running his fingers gently through the dog’s soft brown fur. Sherlock (in green) uses his other hand to point toward the sky whil

### Split the scenes using regex. The regex will remove all the scense..segment.. and split on that. Then join by stepping on 2 and make a scene line

In [33]:
pattern = re.compile(r'(Scene\s+\d+\s+Segment\s+\d+)')
parts_with_delimiter = pattern.split(scene_part)
scene_lines = []
for i in range(1, len(parts_with_delimiter), 2):
    scene_lines.append(parts_with_delimiter[i] + parts_with_delimiter[i+1])

There are 10 scenes and each scene has two segments. We have to gen a image for each segment and store it in a dict with prompt and img, the both seg images are then put in an array an appended to the scene images array. 

In [44]:

scene_images = []
for i, line in enumerate(scene_lines[0:20], start=1):
        # print(line)
        sceneno_desc = line.split(":")
        sceneno = sceneno_desc[0].strip()
        description = sceneno_desc[1].strip()
        # scene_images['scene']
        # # Parse the description to get characters and scene
        parts = description.strip().split(']')
        # names = parts[0].strip('[').split(',')
        scene = parts[2].strip()
        
        # # Create prompt for image generation
        prompt = "stock illustration style, minimalist style, " + scene
        # available_image_url = []
        # # Generate images twice for better selection
        # # for j in range(2):
        #     # Try generating until getting image without black margins
        #     # while True:
        #         # try:
        #             # Generate 4 images for each attempt
            
        #     # for k in range(4):
        #         # Generate image using Stable Diffusion
        res = fal.prompt(prompt)
        #         # Upload to Imgur
        scene_images.append({'scene':sceneno,'prompt':prompt,'image_url':res['images'][0]['url']})

                    # Check for black margins
                    # if not processor.has_black_borders(save_path):
                    #     break
                        
                # except Exception as e:
                #     print(f"Error: {e}. Regenerating...")
                #     continue

 64%|██████▍   | 18/28 [00:01<00:00, 13.77it/s]


In [58]:
sample_scene_images = [{"scene": 'Scene 1 Segment 1',
  'prompt': "stock illustration style, minimalist style, .\nIsaac (in white) sits on the light brown ground under the large green tree, resting Isaac's back against the solid central trunk. Isaac (in white) reaches out toward the red round apple lying on the light brown ground beside Isaac (in white). Above Isaac (in white), the large green tree's round leafy crown sways gently.",
  'image_url': 'https://v3.fal.media/files/koala/Kv-CuTy-th-6221_XlQ81.png'},
 {'scene': 'Scene 1 Segment 2',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) sits on the light brown ground under the large green tree with the red round apple in Isaac’s (in white) hands. Isaac (in white) closely inspects the red round apple while the bright green leaves of the large green tree sparkle above Isaac (in white).',
  'image_url': 'https://v3.fal.media/files/monkey/gtMR7NBFw6iy796HkigXI.png'},
 {'scene': 'Scene 2 Segment 1',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) stands in the light brown clearing beside the large green tree, pacing back and forth. Isaac (in white) glances repeatedly at a spot on the light brown ground where the red round apple fell earlier.',
  'image_url': 'https://v3.fal.media/files/lion/6NGCYpaATylaTVSrAefGs.png'},
 {'scene': 'Scene 2 Segment 2',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) crouches beneath the large green tree, extending one hand to touch the light brown soil nearby. Isaac’s (in white) expression shows deep focus as Isaac (in white) examines the ground.',
  'image_url': 'https://v3.fal.media/files/panda/GhcMORSAaTC4mdOv98rXw.png'},
 {'scene': 'Scene 3 Segment 1',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) kneels beside the calm blue pond, holding several small gray round pebbles in Isaac’s (in white) hand. Isaac (in white) tosses one gray round pebble into the still blue water, which forms outward ripples across the pond.',
  'image_url': 'https://v3.fal.media/files/tiger/B613ZBdeQ1A65E86OaoG3.png'},
 {'scene': 'Scene 3 Segment 2',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) sits on the soft grassy edge of the calm blue pond, tossing additional small gray round pebbles into the still blue water. Isaac’s (in white) gaze is fixed on the spreading ripples created by the gray round pebbles.',
  'image_url': 'https://v3.fal.media/files/penguin/dOC0ogw4Fip_IiVNgn4wt.png'},
 {'scene': 'Scene 4 Segment 1',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) sits cross-legged on the soft grassy edge of the calm blue pond. Isaac (in white) holds one of the small gray round pebbles, observing its size and weight with focused attention.',
  'image_url': 'https://v3.fal.media/files/rabbit/A9QKXQjrmuurkKY9lTVyc.png'},
 {'scene': 'Scene 4 Segment 2',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) tosses another small gray round pebble into the calm blue pond, watching as intricate ripples form across the still blue water. Isaac’s (in white) thoughtful facial expression reflects determination.',
  'image_url': 'https://v3.fal.media/files/koala/6h3xOi66oRPIG5I7MOPsP.png'},
 {'scene': 'Scene 5 Segment 1',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) sits at a simple wooden table under the dark navy sky, the table holding an open brown rectangular book. Isaac (in white) dips a quill into an inkwell and writes on the pages of the brown rectangular book, illuminated by starlight.',
  'image_url': 'https://v3.fal.media/files/kangaroo/Jm-EE7UoeLo-L1jphaxmP.png'},
 {'scene': 'Scene 5 Segment 2',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) pauses, leaning forward over the wooden table as Isaac (in white) scribbles equations into the brown rectangular book. Bright white stars appear scattered in the dark navy sky above Isaac (in white).',
  'image_url': 'https://v3.fal.media/files/tiger/rX5mDTWIQagYHFENn3cfk.png'},
 {'scene': 'Scene 6 Segment 1',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) stands behind the wooden table, flipping through the pages of the brown rectangular book resting on the table. Isaac’s (in white) oval face is softly lit by the glow of the stars above.',
  'image_url': 'https://v3.fal.media/files/tiger/aGt4WSYIzkV91-Be73_4X.png'},
 {"scene": 'Scene 6 Segment 2',
  'prompt': "stock illustration style, minimalist style, .\nIsaac (in white) presses both hands against the wooden table, a wide smile on Isaac's (in white) face. The starlit sky casts a soft glow on Isaac’s (in white) figure and the brown rectangular book.",
  'image_url': 'https://v3.fal.media/files/lion/nHZWE2ZSndKTNnw2fzcEM.png'},
 {'scene': 'Scene 7 Segment 1',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) leans over the wooden table at the starlit workspace, sketching precise lines and curves in the brown rectangular book. The quill in Isaac’s (in white) hand moves across the pages with focus.',
  'image_url': 'https://v3.fal.media/files/penguin/DQ8cGuGgRmV1DKKJo4LPz.png'},
 {'scene': 'Scene 7 Segment 2',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) straightens slightly while reviewing the brown rectangular book under the night sky, which sparkles with countless bright white stars above Isaac (in white).',
  'image_url': 'https://v3.fal.media/files/monkey/d4Q9PNQWjeiMqVlRKLDk2.png'},
 {'scene': 'Scene 8 Segment 1',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) steps back from the wooden table, holding the brown rectangular book in both hands. Isaac’s (in white) gaze is directed toward the vast starry sky above, which stretches endlessly.',
  'image_url': 'https://v3.fal.media/files/panda/UnZ1u6W2dmwNFUqiu5S6k.png'},
 {'scene': 'Scene 8 Segment 2',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) cradles the brown rectangular book with pride. The starlight reflects in Isaac’s (in white) thoughtful brown eyes as Isaac (in white) looks upward.',
  'image_url': 'https://v3.fal.media/files/monkey/9gsCx34zo1WbjW82o_bU-.png'},
 {'scene': 'Scene 9 Segment 1',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) stands beneath the large green tree while holding the brown rectangular book under Isaac’s (in white) arm. Isaac’s (in white) gaze is directed at the bright green round leafy crown of the large green tree.',
  'image_url': 'https://v3.fal.media/files/penguin/YdPVl-LL9K0Fc-MVvaJMG.png'},
 {'scene': 'Scene 9 Segment 2',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) continues to look upward at the rustling bright green leaves of the large green tree’s crown, the brown rectangular book tucked securely under Isaac’s (in white) arm.',
  'image_url': 'https://v3.fal.media/files/kangaroo/ntktk3wOGARI5KUtGSw3S.png'},
 {"scene": 'Scene 10 Segment 1',
  'prompt': "stock illustration style, minimalist style, .\nIsaac (in white) sits at the wooden table under the sparkling starry sky. Isaac's (in white) quill moves carefully across the open page of the brown rectangular book.",
  'image_url': 'https://v3.fal.media/files/kangaroo/cF7Zm-vXPuSB-kovcbd2p.png'},
 {'scene': 'Scene 10 Segment 2',
  'prompt': 'stock illustration style, minimalist style, .\nIsaac (in white) sets the quill down and holds the brown rectangular book. Isaac’s (in white) eyes reflect the glow of the star-filled sky above as Isaac (in white) gazes at the night in quiet accomplishment.',
  'image_url': 'https://v3.fal.media/files/penguin/PEMCk7hjmvABZlEb_8aCC.png'}]

In [45]:

video = story
mp4_files =[]
id+=1
for i in range(segment_num):
    item = scene_images[i]
    sceneno = item['scene']
    prompt = item['prompt'] 
    prompt = prompt[:10]
    prompts = [[prompt]]
    image = item['image_url']
    res = vidclient.prompt(image)
    save_path = os.path.join(f'code/result/',f'video_{id}/')
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    url = res['video']['url']
    with urlopen(url) as response:
        # Open the local file in binary write mode
        with open(os.path.join(f'{save_path}/', f"{sceneno.replace(' ', '_')}.mp4"), 'wb') as video_file:
            # Write the video content to file
            video_file.write(response.read())
    mp4_files.append(f"{sceneno.replace(' ', '_')}.mp4")
    print(res)

100%|██████████| 4/4 [00:05<00:00,  1.33s/it]
{'video': {'url': 'https://fal.media/files/panda/xaelb7_CelRFP3R5RBRFc.mp4', 'content_type': 'video/mp4', 'file_name': 'reenc-tmptzhrsvaq.mp4', 'file_size': 196126}, 'seed': 6789159743605580490}
100%|██████████| 4/4 [00:05<00:00,  1.32s/it]
Inference took: 7.626242975704372s, uploading the result...
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3l

FalClientError: User is locked. Reason: Exhausted balance. Top up your balance at fal.ai/dashboard/billing.

In [61]:


file_list_path = os.path.join(f'code/result/video_{id}/', 'file_list.txt')
with open(file_list_path, 'w') as file_list:
    for mp4_file in mp4_files:
        file_list.write(f"file '{mp4_file}'\n")
print('done1')
output_path = os.path.join('./output/', f"{video.replace(' ','_')}.mp4")
print('done2')
ffmpeg_command = [
    'ffmpeg',
    '-f', 'concat',
    '-safe', '0',
    '-i', file_list_path,
    '-c', 'copy',
    output_path
]

subprocess.run(ffmpeg_command, check=True)

done1
done2


ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1_3 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

CompletedProcess(args=['ffmpeg', '-f', 'concat', '-safe', '0', '-i', 'code/result/video/file_list.txt', '-c', 'copy', './output/story_of_how_calculus_was_invented.mp4'], returncode=0)